In [1]:
%load_ext autoreload
%autoreload 2


In [2]:

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoModel
import transformers
import torch

model = "tiiuae/falcon-7b"

tokenizer = AutoTokenizer.from_pretrained(model)

/opt/conda/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:


from utils import latkar, karlat, get_bible

ge_bible = get_bible()
en_bible = karlat(ge_bible)

print(ge_bible[:30], "<->", en_bible[:30])

 თავდაპირველაღ ღმერთმა შექმნა  <->  tavdap’irvelagh ghmertma shek


In [4]:
import re
# Split on numbering (e.g 1., 2., 3., )
ls = re.split(r"\d+\.", ge_bible)
ls[:20]

[' თავდაპირველაღ ღმერთმა შექმნა ცა და მიწა. ',
 ' მიწა იყო უსახო და უდაბური, ბნელი იდო უფსკრულზე და სული ღვთისა იძვროდა წყლებს ზემოთ. ',
 ' თქვა ღმერთმა: იყოს ნათელი! და იქმნა ნათელი. ',
 ' და ნახა ღმერთმა, რომ ნათელი კარგი იყო, და გაჰყარა ღმერთმა ნათელი და ბნელი. ',
 ' ნათელს ღმერთმა უწოდა დღე და ბნელს უწოდა ღამე. იყო საღამო, იყო დილა პირველი დღე. ',
 ' თქვა ღმერთმა: იყოს წყალთა შორის მყარი და გაჰყაროს წყლები. ',
 ' გააჩინა ღმერთმა მყარი და გაჰყარა ერთმანეთისგან წყალი, რომელიც არის მყარს ქვემოთ, და წყალი, რომელიც არის მყარს ზემოთ. და იქმნა ასე. ',
 ' მყარს ღმერთმა უწოდა ცა. იყო საღამო, იყო დილა - მეორე დღე.  ',
 ' თქვა ღმერთმა: შეგროვდეს ერთგან ცისქვეშეთის წყალი და გამოჩნდეს ხმელეთი. და იქმნა ასე. ',
 ' ხმელეთს ღმერთმა უწოდა მიწა და შეგროვილ წყალს უწოდა ზღვა. დაინახა ღმერთმა, რომ კარგი იყო. 11 . თქვა ღმერთმა: აღმოაცენოს მიწამ მცენარეული - ბალახი, თესლის მთესველი, ხე ნაყოფიერი, თესლოვანი ნაყოფის მომტანი მიწაზე თავისი გვარისდა მიხედვით. და იქმნა ასე. ',
 ' წარმოშვა მიწამ მცენარეული - ბა

In [5]:
from datasets import load_dataset, Dataset

ls = [karlat(s) for s in ls]
dataset = Dataset.from_dict({"text": ls})
dataset = dataset.train_test_split(test_size=0.1)
print(dataset)
dataset["train"][0]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 33642
    })
    test: Dataset({
        features: ['text'],
        num_rows: 3738
    })
})


{'text': ' utkhra samuelma mtels israels: aha, q’velaperi mogisminet, rasats moitkhovdit chemgan da dagisvit mepe. '}

In [6]:
def chunk_examples(examples):
    chunks = []
    for sentence in examples["text"]:
        chunks += [sentence[i:i + 100] for i in range(0, len(sentence), 100)]
    return {"text": chunks}

def preprocess_function(examples):
    r =  examples
    # r = 
    # assert isinstance(r, str), r
    # r = karlat(r)
    out = tokenizer(r['text'], truncation=True, max_length=100)
    # for ex in examples["text"]:
    #     r = tokenizer(ex)
    return out

dataset = dataset.map(
    chunk_examples,
    batched=True,
    num_proc=1,
)

dataset = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
)
dataset

Map:   0%|          | 0/33642 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 54893
    })
    test: Dataset({
        features: ['text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6058
    })
})

In [7]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer
from transformers import DataCollatorForLanguageModeling

model = AutoModelForCausalLM.from_pretrained(
    "tiiuae/falcon-7b", 
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)


==============================WARNING: DEPRECATED!==============================
WARNING! This version of bitsandbytes is deprecated. Please switch to `pip install bitsandbytes` and the new repo: https://github.com/TimDettmers/bitsandbytes
==============================WARNING: DEPRECATED!==============================


RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
module 'bitsandbytes.nn' has no attribute 'Linear8bitLt'

In [ ]:

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    report_to='none',
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    # bf16=True,
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
)

trainer.train()


/opt/conda/envs/torch/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


AttributeError: 'function' object has no attribute '__func__'

In [ ]:
block_size = 128


def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:

en = tokenizer.encode(
    ge_bible[:38]
)
dec = tokenizer.decode(en)
print("GE bible", len(en), dec, en)


en = tokenizer.encode(
    en_bible[:35]
)
dec = tokenizer.decode(en)
print("EN bible", len(en), dec, en)

In [ ]:
from transformers import top_k_top_p_filtering

In [ ]:
inputs = tokenizer.encode(en_bible[:35], return_tensors='pt')
inputs.shape

In [ ]:
outputs = model(inputs)

In [ ]:
outputs.logits.shape

In [ ]:
import torch.nn.functional as F

In [ ]:
last_layer_logits = outputs.logits[:, -1, :]

# Keep top 30 logits at max; stop if cumulative probability >= 1.0.
top_logits = top_k_top_p_filtering(last_layer_logits, top_k=100, top_p=1.0)

# Softmax the logits into probabilities
probabilities = F.softmax(top_logits, dim=-1)

# Generate next token
generated_next_token = torch.multinomial(probabilities, num_samples=1)
generated = torch.cat([inputs, generated_next_token], dim=-1)

In [ ]:
generated

In [ ]:
# Get result
result_string = tokenizer.decode(generated.tolist()[0])

In [ ]:
from datasets import load_dataset

ge_bible[:100]

In [ ]:
print(latkar(en_bible[:38]), '->', latkar(result_string), 'actual', latkar(ge_bible[:40]))
print(en_bible[:38], '->', result_string, 'actual', ge_bible[:40])

In [ ]:

# pipeline = transformers.pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     torch_dtype=torch.bfloat16,
#     trust_remote_code=True,
#     device_map="auto",
# )
# sequences = pipeline(
#    "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\nDaniel: Hello, Girafatron!\nGirafatron:",
#     # max_length=200,
#     # do_sample=True,
#     # top_k=10,
#     # num_return_sequences=1,
#     # eos_token_id=tokenizer.eos_token_id,
# )
model